#### Assinging top100 evidences to claims

In [ ]:
%load_ext autoreload
%autoreload

import json
import os
from src.common import DATA_PATH, read_json, QT_VERACITY_LABELS


def process(claims_file, evidences_file):
    claims = read_json(os.path.join(DATA_PATH, claims_file))
    evidences = read_json(os.path.join(DATA_PATH, evidences_file))

    evidences = {e['claim']: e for e in evidences}

    for claim in claims:
        claim['top100evidences'] = evidences[claim['claim']]['top100evidences']

    print(len(claims), len(evidences))
    save_json(f'processed_{claims_file}', claims)


process(f'{DATA_PATH}/raw_data/train_claims_quantemp.json', f'{DATA_PATH}/train_evidences.json')
process(f'{DATA_PATH}/raw_data/val_claims_quantemp.json', f'{DATA_PATH}/val_evidences.json')
process(f'{DATA_PATH}/raw_data/test_claims_quantemp.json', f'{DATA_PATH}/test_evidences.json')

#### Fixing format for gpt3.5-turbo decomposition

In [9]:
%load_ext autoreload
%autoreload

import json
import pandas as pd
from src.common import DATA_PATH, read_json

claims = read_json(f'{DATA_PATH}/raw_data/test_claims_100_evidences.json')
decomposed_questions = pd.read_csv(f'{DATA_PATH}/test_claimdecomp.csv', sep="@")

for claim in claims:#
    questions = decomposed_questions[decomposed_questions['claims'] == claim['claim']]['questions']

    if len(questions) == 0:
        questions = []
    elif len(questions) == 1:
        questions = questions.iloc[0].split("Next Question: ")
    else:
        print("ERROR")

    questions = [q.strip() for q in questions]
    claim['questions'] = questions


with open(f'{DATA_PATH}/test_decomposed_gpt3.5-turbo.json', "w") as file:
    json.dump(claims, file, indent=4)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Fixing label names in gpt3.5-turbo decomposition

In [18]:
%load_ext autoreload
%autoreload

import json
import os
from src.common import DATA_PATH, read_json, save_json, QT_VERACITY_LABELS

def fix_labels(filename):
    claims = read_json(os.path.join(DATA_PATH, filename))

    for claim in claims:
        if claim['label'] == "Half True/False":
            claim['label'] = "Conflicting"

        assert claim['label'] in QT_VERACITY_LABELS

    print(len(claims))
    save_json(os.path.join(DATA_PATH, f'fixed_{filename}'), claims)


fix_labels('train_evidences_decomposed_gpt3.5-turbo.json')
fix_labels('val_evidences_decomposed_gpt3.5-turbo.json')
fix_labels('test_evidences_decomposed_gpt3.5-turbo.json')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
9935
3084
2495


#### Finding top evidences for decomposed questions / claim

In [8]:
%load_ext autoreload
%autoreload

from src.common import read_json, DATA_PATH, save_json
from src.evidence_processor import EvidenceProcessor

processor = EvidenceProcessor(decomposed=True, top_k=1)

test_dataset = read_json(f'{DATA_PATH}/test_decomposed_gpt3.5-turbo.json')

test_dataset_evidence = processor.transform(test_dataset)

save_json(f'{DATA_PATH}/test_evidences_decomposed_gpt3.5-turbo.json', test_dataset_evidence)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


  0%|          | 0/2495 [00:00<?, ?it/s]